# Homework 5

#### Author: Gustavo Soares

##### Imports

In [1]:
import pandas as pd
import numpy as np
from dataapi import DBConnect, SGS
import matplotlib.pyplot as plt
from calendars import DayCounts
from tqdm import tqdm
%matplotlib inline

### Dados do DI Futuro
Servidor AWS do FinanceHub.

In [6]:
dbc = DBConnect(username='fhreadonly', password='finquant')
query = "select * from \"B3futures\" where contract='DI1'"
df = pd.read_sql(query, dbc.connection)
df.tail(10)

,time_stamp,contract,maturity_code,open_interest_open,open_interest_close,number_of_trades,trading_volume,financial_volume,previous_settlement,indexed_settlement,opening_price,minimum_price,maximum_price,average_price,last_price,settlement_price,last_bid,last_offer
150498,2020-05-15,DI1,N25,11145,11145,49,455,32126951,70570.16,70825.46,7.04,7.03,7.12,7.051,7.10,70817.43,0.00,7.38
150499,2020-05-15,DI1,F26,71438,71438,134,1005,67444209,66976.28,67171.02,7.39,7.28,7.42,7.345,7.37,67163.40,7.32,0.00
150500,2020-05-15,DI1,N26,9425,9425,13,125,7991530,63829.45,64047.82,7.59,7.59,7.60,7.594,7.59,64040.56,0.00,0.00
150501,2020-05-15,DI1,F27,388313,388313,3186,33825,2053672777,60517.49,60759.03,7.93,7.75,7.95,7.830,7.87,60752.14,7.86,7.90
150502,2020-05-15,DI1,F28,25475,25475,43,240,13156569,54553.74,54705.47,8.25,8.12,8.26,8.213,8.25,54699.27,0.00,0.00
150503,2020-05-15,DI1,F29,201417,201417,1439,13170,654038139,49635.17,49690.60,8.54,8.40,8.59,8.476,8.54,49684.97,8.51,8.54
150504,2020-05-15,DI1,F30,9045,9045,0,0,0,45002.15,45096.58,0.00,0.00,0.00,0.000,0.00,45091.47,0.00,0.00
150505,2020-05-15,DI1,F31,31253,31253,117,815,33126958,40695.92,40828.74,8.86,8.80,8.92,8.864,8.88,40824.11,8.00,9.30
150506,2020-05-15,DI1,F33,20,20,0,0,0,33581.40,33709.63,0.00,0.00,0.00,0.000,0.00,33705.81,0.00,0.00
150507,2020-05-15,DI1,F35,0,0,0,0,0,27761.00,27882.22,0.00,0.00,0.00,0.000,0.00,27879.06,0.00,0.00


### Cálculo dos dias úteis até o vencimento de um contrato

In [7]:
df['time_stamp'] = pd.to_datetime(df['time_stamp'])

mat_dict = {'01': 'F',
            '02': 'G',
            '03': 'H',
            '04': 'J',
            '05': 'K',
            '06': 'M',
            '07': 'N',
            '08': 'Q',
            '09': 'U',
            '10': 'V',
            '11': 'X',
            '12': 'Z'}

mat_dict = {v: k for k, v in mat_dict.items()}

mat_month = df['maturity_code'].str[0].map(mat_dict)

def convert_year(year2digits):
    
    if int(year2digits) >= 91:
        year4digits = '19' + year2digits
    else:
        year4digits = '20' + year2digits
    
    return year4digits

mat_year = df['maturity_code'].str[-2:].apply(convert_year)

dates = pd.to_datetime(mat_year + mat_month + '01')

dc = DayCounts('BUS/252', calendar='anbima')
dates = dc.busdateroll(dates, 'following')
dates

df['maturity_date'] = dates

df['DU'] = dc.days(df['time_stamp'], df['maturity_date'])
df = df[df['DU'] >= 0]

# Utilizando apenas PUs não nulos
df = df[df.settlement_price != 0]

# Apenas Open Interest > 1
df = df[df['trading_volume'] != 0]

df.tail(35)

,time_stamp,contract,maturity_code,open_interest_open,open_interest_close,number_of_trades,trading_volume,financial_volume,previous_settlement,indexed_settlement,opening_price,minimum_price,maximum_price,average_price,last_price,settlement_price,last_bid,last_offer,maturity_date,DU
150470,2020-05-15,DI1,M20,1100495,1100495,28,52520,5245446113,99875.28,99886.49,2.905,2.900,2.905,2.901,2.902,99875.16,2.901,2.902,2020-06-01,11
150471,2020-05-15,DI1,N20,4056567,4056567,334,120635,12021735819,99652.86,99665.06,2.775,2.760,2.785,2.768,2.760,99653.76,2.760,2.775,2020-07-01,32
150472,2020-05-15,DI1,Q20,336540,336540,58,10555,1049520314,99422.48,99446.93,2.640,2.627,2.650,2.637,2.627,99435.65,2.627,0.000,2020-08-03,55
150473,2020-05-15,DI1,U20,141165,141165,119,28925,2870138866,99216.14,99240.13,2.610,2.590,2.630,2.606,2.600,99228.88,2.600,2.670,2020-09-01,76
150474,2020-05-15,DI1,V20,816069,816069,537,55320,5477993893,99006.75,99037.38,2.580,2.560,2.610,2.581,2.570,99026.15,2.570,2.575,2020-10-01,97
150475,2020-05-15,DI1,X20,111820,111820,15,710,70153673,98785.97,98823.27,2.600,2.560,2.600,2.594,2.575,98812.06,2.560,2.610,2020-11-03,118
150476,2020-05-15,DI1,Z20,68885,68885,68,4825,475831895,98576.21,98620.66,2.605,2.550,2.615,2.573,2.585,98609.48,2.545,2.590,2020-12-01,138
150477,2020-05-15,DI1,F21,3713326,3713326,3432,268390,26409277910,98360.14,98419.03,2.720,2.545,2.730,2.574,2.570,98407.87,2.565,2.570,2021-01-04,160
150478,2020-05-15,DI1,G21,67320,67320,8,10615,1042037343,98112.31,98179.39,2.640,2.585,2.640,2.624,2.635,98168.26,2.600,2.625,2021-02-01,180
150479,2020-05-15,DI1,H21,52957,52957,50,2185,214015401,97899.10,97974.05,2.680,2.640,2.690,2.674,2.645,97962.94,2.615,2.650,2021-03-01,198


### Série do CDI de um dia
API do [Sistema Gerenciador de Series](https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries) do BCB.

In [8]:
sgs = SGS()
df_cdi = sgs.fetch({12: 'CDI'})/100
df_cdi.tail(5)

,CDI
data,
2020-11-17,0.000075
2020-11-18,0.000075
2020-11-19,0.000075
2020-11-20,0.000075
2020-11-23,0.000075


# Estratégias no futuro de DI-1 

## Q1: Carry

Usando o que você aprendeu sobre o sinal de carry em [Koijen, Moskowitz, Pedersen, and Vrugt (2016)](https://ssrn.com/abstract=2298565) e [Brooks and Moskowitz (2017)](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2956411), faça o cálculo do carry anualizado esperado de se segurar um contrato de futuros DI-1 por 3 meses. Calcule o valor do carry para todos os contratos e todos os dias do dataframe ```df``` acima. Calcule o carry do contrato "puro" e também do carry de 3 meses ajustado pelo DV01 do contrato.

## Q2: Carry trade

Usando o que você aprendeu sobre carry portfolios em [Koijen, Moskowitz, Pedersen, and Vrugt (2016)](https://ssrn.com/abstract=2298565) e [Brooks and Moskowitz (2017)](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2956411), faça o backtest de uma estratégia DV01 neutra que escolhe contratos de acordo com seu carry para criar DV01 flattners ou steepners. A estratégia final deve ser DV01 neutra (ou seja, ajuste os holdings por DV01 nas datas de rebalanceamento das posições). Não transacionar contratos que tem zero open interest ou zero volume.

## Q3: Market timing

Usando o que você aprendeu sobre carry timing em [Koijen, Moskowitz, Pedersen, and Vrugt (2016)](https://ssrn.com/abstract=2298565), faça o backtest de uma estratégia direcional que fica "tomada" ou "dada" em DI-1 de acordo com o sinal de carry. Faça com que seus trades tenham sempre o mesmo DV01 como percentual do notional investido, i.e., todo trade tenha o mesmo duration. Compare o resultado com estar constatemente "dado" no mesmo duration. Não transacionar contratos que tem zero open interest ou zero volume.